# Tracing Basics

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [2]:
# You can set them inline
import os

In [3]:
# Or you can use a .env file
#from dotenv import load_dotenv
#load_dotenv(dotenv_path="../.env", override=True)

### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [6]:
# TODO: Import traceable
from typing import List
import nest_asyncio
#from llmhelper import get_llm, get_retriever
from common_genai_utils.geminihelper import get_llm
from common_genai_utils.llmhelper import get_retriever

APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

llm_client = get_llm()
nest_asyncio.apply()
retriever = get_retriever(llm_model=llm_client)

# TODO: Set up tracing for each function
def retrieve_documents(question: str):
    return retriever.invoke(question)   # NOTE: This is a LangChain vector db retriever, so this .invoke() call will be traced automatically


def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_model(messages)


def call_model(
    messages: List[dict]
) -> str:
    return llm_client.invoke(messages)


def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response


USER_AGENT environment variable not set, consider setting it to identify your requests.


model name used -  models/gemini-1.5-pro


@traceable handles the RunTree lifecycle for you!

In [7]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

ResponseError: model "models/gemini-1.5-pro" not found, try pulling it first

##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [11]:
from langsmith import traceable

@traceable(
    # TODO: Add Metadata
    metadata={"vectordb": "Chroma DB"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

MODEL_NAME = llm_client.model
MODEL_PROVIDER = llm_client.model

@traceable(
    # TODO: Add Metadata
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_openai(
    messages: List[dict]
) -> str:
    return llm_client.invoke(messages)

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response


In [12]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

I'm not aware of specific information on how to add metadata to a run using @traceable. However, I can suggest checking the documentation or support resources for @traceable for more detailed guidance. If available, it's likely that you'll find instructions on how to integrate metadata with your specific use case.


You can also add metadata at runtime!

In [13]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

I don't have specific information on adding metadata at runtime. However, I can suggest that it depends on the programming language or framework being used, as different languages have varying approaches to dynamic metadata management. Could you please provide more context or specify which language or framework you're working with?


##### Let's take a look in LangSmith!